In [5]:
import numpy as np
import sys
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV

from pointcloud import PointCloud
import pandas as pd
import matplotlib.pyplot as plt

from train import split_data, train_svm_linear, train_svm_rbf, train_svm_poly, train_random_forest

## Build dataset and split in train and test

In [2]:
labels = ["building", "car", "fence", "pole", "tree"]
labels_dataset = []
features_dataset = []
for hundred in range(5):
    class_name = labels[hundred]
    print(f"Processing {class_name}")
    for i in range(100): #todo 100
        if i % 10 == 0:
            print(f"Processing {class_name}: {i}")
        point_cloud = PointCloud.from_file(f"data/pointclouds/{hundred}{i:02d}.xyz")
        features = point_cloud.compute_feature_matrix()
        labels_dataset.append(class_name)
        features_dataset.append(features)

train_features, test_features, train_labels, test_labels = split_data(features_dataset, labels_dataset)

Processing building
Processing building: 0
Processing building: 10
Processing building: 20
Processing building: 30
Processing building: 40
Processing building: 50
Processing building: 60
Processing building: 70
Processing building: 80
Processing building: 90
Processing car
Processing car: 0
Processing car: 10
Processing car: 20
Processing car: 30
Processing car: 40
Processing car: 50
Processing car: 60
Processing car: 70
Processing car: 80
Processing car: 90
Processing fence
Processing fence: 0
Processing fence: 10
Processing fence: 20
Processing fence: 30
Processing fence: 40
Processing fence: 50
Processing fence: 60
Processing fence: 70
Processing fence: 80
Processing fence: 90
Processing pole
Processing pole: 0
Processing pole: 10
Processing pole: 20
Processing pole: 30
Processing pole: 40
Processing pole: 50
Processing pole: 60
Processing pole: 70
Processing pole: 80
Processing pole: 90
Processing tree
Processing tree: 0
Processing tree: 10
Processing tree: 20
Processing tree: 30
P

## Train SVMs with different Kernels
The functions use grid search to find the best parameters for the SVMs.

In [3]:
svm_linear = train_svm_linear(train_features, train_labels)
svm_poly = train_svm_poly(train_features, train_labels)
svm_rbf = train_svm_rbf(train_features, train_labels)
rf_model = train_random_forest(train_features, train_labels)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Found best parameters for poly kernel:
{'C': 10000000000, 'degree': 2, 'kernel': 'poly'}
[CV] END .......................C=0.1, degree=2, kernel=poly; total time=   0.0s
[CV] END .......................C=0.1, degree=5, kernel=poly; total time=   0.0s
[CV] END ......................C=1000, degree=4, kernel=poly; total time=   0.0s
[CV] END ......................C=1000, degree=4, kernel=poly; total time=   0.6s
[CV] END ......................C=1000, degree=4, kernel=poly; total time=   0.1s
[CV] END ......................C=1000, degree=4, kernel=poly; total time=   0.0s
[CV] END ...................C=1000000, degree=2, kernel=poly; total time=   1.0s
[CV] END ...................C=1000000, degree=2, kernel=poly; total time=   4.6s
[CV] END ...................C=1000000, degree=2, kernel=poly; total time=   0.5s
[CV] END ...................C=1000000, degree=3, kernel=poly; total time=   4.4s
[CV] END .................C=100000000, 

## Print results of SVMs

In [4]:
y_pred_linear = svm_linear.predict(test_features)
print("Linear Kernel:")
print(confusion_matrix(test_labels, y_pred_linear))
print(classification_report(test_labels, y_pred_linear))

# Polynomial kernel
y_pred_poly = svm_poly.predict(test_features)
print("Polynomial Kernel:")
print(confusion_matrix(test_labels, y_pred_poly))
print(classification_report(test_labels, y_pred_poly))

y_pred_rbf = svm_rbf.predict(test_features)
print("RBF Kernel:")
print(confusion_matrix(test_labels, y_pred_rbf))
print(classification_report(test_labels, y_pred_rbf))

y_pred_rf = rf_model.predict(test_features)
print("Random Forest:")
print(confusion_matrix(test_labels, y_pred_rf))
print(classification_report(test_labels, y_pred_rf))

Linear Kernel:
[[27  0  0  0  1]
 [ 0 14  0  0  0]
 [ 0  0 10  0  0]
 [ 0  1  0 22  1]
 [ 1  0  0  0 23]]
              precision    recall  f1-score   support

    building       0.96      0.96      0.96        28
         car       0.93      1.00      0.97        14
       fence       1.00      1.00      1.00        10
        pole       1.00      0.92      0.96        24
        tree       0.92      0.96      0.94        24

    accuracy                           0.96       100
   macro avg       0.96      0.97      0.97       100
weighted avg       0.96      0.96      0.96       100

Polynomial Kernel:
[[28  0  0  0  0]
 [ 0 14  0  0  0]
 [ 0  0 10  0  0]
 [ 0  1  0 23  0]
 [ 1  0  0  0 23]]
              precision    recall  f1-score   support

    building       0.97      1.00      0.98        28
         car       0.93      1.00      0.97        14
       fence       1.00      1.00      1.00        10
        pole       1.00      0.96      0.98        24
        tree       1.00 